In [2]:
import pandas as pd
import os
import re

In [3]:
full_path = "/app/nse/ml/"
wrong_data_folder = "output_28_12_2024/"

In [7]:
target_full_pd = pd.read_csv(full_path + wrong_data_folder + "full_target.csv")
is_unnamed = pd.isna(target_full_pd.columns[0]) or str(target_full_pd.columns[0]).startswith('Unnamed:')
if is_unnamed:
    target_full_pd = target_full_pd.drop(target_full_pd.columns[0], axis=1)

features_full_pd = pd.read_csv(full_path + wrong_data_folder + "full_features.csv")
is_unnamed = pd.isna(features_full_pd.columns[0]) or str(features_full_pd.columns[0]).startswith('Unnamed:')
if is_unnamed:
    features_full_pd = features_full_pd.drop(features_full_pd.columns[0], axis=1)

features_pd = pd.read_csv(full_path + wrong_data_folder + "features.csv")
is_unnamed = pd.isna(features_pd.columns[0]) or str(features_pd.columns[0]).startswith('Unnamed:')
if is_unnamed:
    features_pd = features_pd.drop(features_pd.columns[0], axis=1)

In [5]:
right_column_f = ["y", "z", "u", "power", "roughness", "T", "sensible_heat_flux", "T_grad", "distances"]
right_column_t = ["c_mean_z", "c_mean_y", "c_std_z", "c_std_z"]

change_column_f = {"y": "y", "z": "z", "u":"u", "p": "power", "z0_m": "roughness", "T0": "T", 
                   "value": "sensible_heat_flux", "grad_z": "T_grad","distance": "distances"}
change_column_t = {"mean_z": "c_mean_z", "mean_y": "c_mean_y", "std_z": "c_std_z", "std_y": "c_std_y"}

In [9]:
features_pd.columns

Index(['y', 'z', 'u', 'roughness', 'T', 'power', 'sensible_heat_flux',
       'T_grad'],
      dtype='object')

In [8]:
features_pd.rename(columns=change_column_f, inplace=True)
features_full_pd.rename(columns=change_column_f, inplace=True)
target_full_pd.rename(columns=change_column_t, inplace=True)

features_pd_reord = features_pd.iloc[:, [0, 1, 2, 5, 3, 4, 6, 7]]
features_full_pd_reord = features_full_pd.iloc[:, [0, 1, 2, 5, 3, 4, 6, 7, 8]]
target_full_pd_reord = target_full_pd.iloc[:, [1, 0, 3, 2]]

features_pd_reord.to_csv(full_path + wrong_data_folder + "features.csv", index=False)
features_full_pd_reord.to_csv(full_path + wrong_data_folder + "features_full.csv", index=False)
target_full_pd_reord.to_csv(full_path + wrong_data_folder + "target_full.csv", index=False)

In [6]:
files_in_directory = os.listdir(full_path + wrong_data_folder)
file_pattern = r"target_\d"
targets_for_one_dist = [file for file in files_in_directory if re.search(file_pattern, file)]
for file in targets_for_one_dist:
    file_pd = pd.read_csv(full_path + wrong_data_folder+ file)
    file_pd.rename(columns=change_column_t, inplace=True)
    file_pd_reord = file_pd.iloc[:, [2, 0, 3, 1]]
    file_list = file.split("_")
    file_pd_reord.to_csv(full_path + wrong_data_folder + "target-"+file_list[1]+"_"+file_list[2], index=False)
    

In [10]:
target_full_pd["distances"] = features_full_pd["distances"]
distances_list = features_full_pd["distances"].values

In [12]:
target_full_pd["_tmp_index"] = target_full_pd.index
target_pd_sort = target_full_pd.sort_values(by=["distances", "_tmp_index"]) 
target_pd_sort = target_pd_sort.drop(columns=["_tmp_index"])

In [13]:
import numpy as np

In [14]:
for dist in np.unique(distances_list):
    tmp_table = target_pd_sort[target_pd_sort.distances == dist].dropna().drop(columns=["distances"])
    tmp_table.to_csv(full_path + wrong_data_folder + "target-" + str(dist).replace(".", "_") + ".csv", index=False)